In [1]:
import os, sys, time, glob, random, argparse
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn

# XAutoDL 
from xautodl.config_utils import load_config, dict2config, configure2str
from xautodl.datasets import get_datasets, get_nas_search_loaders
from xautodl.procedures import (
    prepare_seed,
    prepare_logger,
    save_checkpoint,
    copy_checkpoint,
    get_optim_scheduler,
)
from xautodl.utils import get_model_infos, obtain_accuracy
from xautodl.log_utils import AverageMeter, time_string, convert_secs2time
from xautodl.models import get_search_spaces

from custom_models import get_cell_based_tiny_net
from custom_search_cells import NAS201SearchCell as SearchCell
from xautodl.models.cell_searchs.genotypes import Structure

# NB201
from nas_201_api import NASBench201API as API

import scipy.stats as stats
from tqdm import tqdm

2022-11-08 04:15:01.214027: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
parser = argparse.ArgumentParser("Random search for NAS.")
parser.add_argument("--data_path", type=str, default='../cifar.python', help="The path to dataset")
parser.add_argument("--dataset", type=str, default='cifar10',choices=["cifar10", "cifar100", "ImageNet16-120"], help="Choose between Cifar10/100 and ImageNet-16.")

# channels and number-of-cells
parser.add_argument("--search_space_name", type=str, default='nas-bench-201', help="The search space name.")
parser.add_argument("--config_path", type=str, default='./MY.config', help="The path to the configuration.")
parser.add_argument("--max_nodes", type=int, default=4, help="The maximum number of nodes.")
parser.add_argument("--channel", type=int, default=16, help="The number of channels.")
parser.add_argument("--num_cells", type=int, default=5, help="The number of cells in one stage.")
parser.add_argument("--select_num", type=int, default=100, help="The number of selected architectures to evaluate.")
parser.add_argument("--track_running_stats", type=int, default=0, choices=[0, 1], help="Whether use track_running_stats or not in the BN layer.")
# log
parser.add_argument("--workers", type=int, default=4, help="number of data loading workers")
parser.add_argument("--save_dir", type=str, default='./arch_level-use_pretrain-acc_metric', help="Folder to save checkpoints and log.")
# parser.add_argument("--arch_nas_dataset", type=str, default='../NAS-Bench-201-v1_1-096897.pth', help="The path to load the architecture dataset (tiny-nas-benchmark).")
parser.add_argument("--arch_nas_dataset", type=str, default=None, help="The path to load the architecture dataset (tiny-nas-benchmark).")
parser.add_argument("--print_freq", type=int, default=200, help="print frequency (default: 200)")
parser.add_argument("--rand_seed", type=int, default=None, help="manual seed")
args = parser.parse_args(args=[])
if args.rand_seed is None or args.rand_seed < 0:
    args.rand_seed = random.randint(1, 100000)

    
print(args.rand_seed)
print(args)
xargs=args

53064
Namespace(arch_nas_dataset=None, channel=16, config_path='./MY.config', data_path='../cifar.python', dataset='cifar10', max_nodes=4, num_cells=5, print_freq=200, rand_seed=53064, save_dir='./arch_level-use_pretrain-acc_metric', search_space_name='nas-bench-201', select_num=100, track_running_stats=0, workers=4)


In [3]:
assert torch.cuda.is_available(), "CUDA is not available."
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_num_threads(xargs.workers)
prepare_seed(xargs.rand_seed)
logger = prepare_logger(args)

Main Function with logger : Logger(dir=arch_level-use_pretrain-acc_metric, use-tf=False, writer=None)
Arguments : -------------------------------
arch_nas_dataset : None
channel          : 16
config_path      : ./MY.config
data_path        : ../cifar.python
dataset          : cifar10
max_nodes        : 4
num_cells        : 5
print_freq       : 200
rand_seed        : 53064
save_dir         : ./arch_level-use_pretrain-acc_metric
search_space_name : nas-bench-201
select_num       : 100
track_running_stats : 0
workers          : 4
Python  Version  : 3.7.13 (default, Mar 29 2022, 02:18:16)  [GCC 7.5.0]
Pillow  Version  : 9.0.1
PyTorch Version  : 1.12.0
cuDNN   Version  : 8302
CUDA available   : True
CUDA GPU numbers : 2
CUDA_VISIBLE_DEVICES : None


In [4]:
train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
config = load_config(xargs.config_path, {"class_num": class_num, "xshape": xshape}, logger)
search_loader, _, valid_loader = get_nas_search_loaders(train_data,
                                                        valid_data,
                                                        xargs.dataset,
                                                        "../configs/nas-benchmark/",
                                                        (config.batch_size, config.test_batch_size),
                                                        xargs.workers)
logger.log("||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}".format(
            xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log("||||||| {:10s} ||||||| Config={:}".format(xargs.dataset, config))

search_space = get_search_spaces("cell", xargs.search_space_name)
model_config = dict2config(
    {
        "name": "RANDOM",
        "C": xargs.channel,
        "N": xargs.num_cells,
        "max_nodes": xargs.max_nodes,
        "num_classes": class_num,
        "space": search_space,
        "affine": False,
        "track_running_stats": bool(xargs.track_running_stats),
    },
    None,
)
search_model = get_cell_based_tiny_net(model_config)

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.parameters(), config)

logger.log("w-optimizer : {:}".format(w_optimizer))
logger.log("w-scheduler : {:}".format(w_scheduler))
logger.log("criterion   : {:}".format(criterion))
# if xargs.arch_nas_dataset is None:
api = None
# else:
#     api = API(xargs.arch_nas_dataset)
logger.log("{:} create API = {:} done".format(time_string(), api))

last_info, model_base_path, model_best_path = (
    logger.path("info"),
    logger.path("model"),
    logger.path("best"),
)
network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()

if last_info.exists():  # automatically resume from previous checkpoint
    logger.log(
        "=> loading checkpoint of the last-info '{:}' start".format(last_info)
    )
    last_info = torch.load(last_info)
    start_epoch = last_info["epoch"]
    checkpoint = torch.load(last_info["last_checkpoint"])
    genotypes = checkpoint["genotypes"]
    valid_accuracies = checkpoint["valid_accuracies"]
    search_model.load_state_dict(checkpoint["search_model"])
    w_scheduler.load_state_dict(checkpoint["w_scheduler"])
    w_optimizer.load_state_dict(checkpoint["w_optimizer"])
    logger.log(
        "=> loading checkpoint of the last-info '{:}' start with {:}-th epoch.".format(
            last_info, start_epoch
        )
    )
else:
    logger.log("=> do not find the last-info file : {:}".format(last_info))
    start_epoch, valid_accuracies, genotypes = 0, {"best": -1}, {}

Files already downloaded and verified
Files already downloaded and verified
./MY.config
Configure(scheduler='cos', LR=0.025, eta_min=0.001, epochs=50, warmup=0, optim='SGD', decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=64, test_batch_size=512, class_num=10, xshape=(1, 3, 32, 32))
||||||| cifar10    ||||||| Search-Loader-Num=391, Valid-Loader-Num=49, batch size=64
||||||| cifar10    ||||||| Config=Configure(scheduler='cos', LR=0.025, eta_min=0.001, epochs=50, warmup=0, optim='SGD', decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=64, test_batch_size=512, class_num=10, xshape=(1, 3, 32, 32))
w-optimizer : SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    initial_lr: 0.025
    lr: 0.025
    maximize: False
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005
)
w-scheduler : CosineAnnealingLR(warmup=0, max-epoch=50, current::epoch=0, iter=0.00, type=cosine, T-max=50, eta-min=0.001)
criterion   : CrossEntropyLoss(

In [5]:
def acc_confidence_robustness_metrics(network, inputs, targets):
    with torch.no_grad():
        # accuracy
        network.train()
        _, logits = network(inputs)
        val_top1, val_top5 = obtain_accuracy(logits.data, targets.data, topk=(1, 5))
        acc = val_top1
        
        return acc.item()
        
#         # confidence
#         prob = torch.nn.functional.softmax(logits, dim=1)
#         one_hot_idx = torch.nn.functional.one_hot(targets)
#         confidence = (prob[one_hot_idx==1].sum()) / inputs.size(0) * 100 # in percent
        
#         # sensitivity
#         _, noisy_logits = network(inputs + torch.randn_like(inputs)*0.1)
#         kl_loss = torch.nn.KLDivLoss(reduction="batchmean")
#         sensitivity = kl_loss(torch.nn.functional.log_softmax(noisy_logits, dim=1), torch.nn.functional.softmax(logits, dim=1))
        
#         # robustness
#         original_weights = deepcopy(network.state_dict())
#         for m in network.modules():
#             if isinstance(m, SearchCell):
#                 for p in m.parameters():
#                     p.add_(torch.randn_like(p) * p.std()*0.3)
            
#         _, noisy_logits = network(inputs)
#         kl_loss = torch.nn.KLDivLoss(reduction="batchmean")
#         robustness = -kl_loss(torch.nn.functional.log_softmax(noisy_logits, dim=1), torch.nn.functional.softmax(logits, dim=1))
#         network.load_state_dict(original_weights)
                
#         return acc.item(), confidence.item(), sensitivity.item(), robustness.item()
    
def step_sim_metric(network, criterion, inputs, targets):
#     inputs, targets = inputs[:64], targets[:64] # smaller batches
    original_dict = deepcopy(network.state_dict())
    optim_large_step = torch.optim.SGD(network.parameters(), lr=0.025)
    
    # single large step
    network.train()
    optim_large_step.zero_grad()
    _, logits = network(inputs)
    base_loss = criterion(logits, targets)
    base_loss.backward()
    optim_large_step.step()
    large_step_dict = deepcopy(network.state_dict())
    
    # multiple small steps
    network.load_state_dict(original_dict)
    optim_small_step = torch.optim.SGD(network.parameters(), lr=0.025/3)
    for i in range(3):
        optim_small_step.zero_grad()
        _, logits = network(inputs)
        base_loss = criterion(logits, targets)
        base_loss.backward()
        optim_small_step.step()
    small_step_dict = deepcopy(network.state_dict())
    scores = []
    for key in large_step_dict.keys():
        if ('weight' in key) and (original_dict[key].dim()==4):
            if (original_dict[key] != large_step_dict[key]).sum():
                large_step = large_step_dict[key] - original_dict[key]
                small_step = small_step_dict[key] - original_dict[key]
                co, ci, kh, kw = large_step.size()
                large_step = large_step.view(co, -1)
                small_step = small_step.view(co, -1)
                score = torch.nn.functional.cosine_similarity(large_step, small_step, dim=1)
                score = score.mean().item() * 100 # in percent
                scores.append(score)
    if len(scores)==0:
        step_sim = 100
        raise RuntimeError
    else:
        step_sim = np.mean(scores)
    
    # resume
    network.load_state_dict(original_dict)
            
    return step_sim

In [6]:
supernet_weights = torch.load("./250ep_cosine_supernet_cell_wise.pth")
network.load_state_dict(supernet_weights)

<All keys matched successfully>

In [7]:
# start training
start_time, search_time, epoch_time, total_epoch = (
    time.time(),
    AverageMeter(),
    AverageMeter(),
    config.epochs + config.warmup,
)

################# initialize
cells = []
for m in network.modules():
    if isinstance(m, SearchCell):
        cells.append(m)
num_cells = len(cells)
print("total number of nodes:{}".format(num_cells*xargs.max_nodes))
        
op_names = deepcopy(cells[0].op_names)
op_names_wo_none = deepcopy(op_names)
if "none" in op_names_wo_none:
    op_names_wo_none.remove("none")

genotypes = []
for i in range(1, xargs.max_nodes):
    xlist = []
    for j in range(i):
        node_str = "{:}<-{:}".format(i, j)
        if i-j==1:
            op_name = "skip_connect"
        else:
            op_name = "none"
        xlist.append((op_name, j))
    genotypes.append(tuple(xlist))
init_arch = Structure(genotypes)

for c in cells:
    c.arch_cache = init_arch

### gen possible connections of a target node
possible_connections = {}
for target_node_idx in range(1,xargs.max_nodes):
    possible_connections[target_node_idx] = list()
    xlists = []
    for src_node in range(target_node_idx):
        node_str = "{:}<-{:}".format(target_node_idx, src_node)
        # select possible ops
#         if target_node_idx - src_node == 1:
#             op_names_tmp = op_names_wo_none
#         else:
#             op_names_tmp = op_names
        op_names_tmp = op_names
            
        if len(xlists) == 0: # initial iteration
            for op_name in op_names_tmp:
                xlists.append([(op_name, src_node)])
        else:
            new_xlists = []
            for op_name in op_names_tmp:
                for xlist in xlists:
                    new_xlist = deepcopy(xlist)
                    new_xlist.append((op_name, src_node))
                    new_xlists.append(new_xlist)
            xlists = new_xlists
    for xlist in xlists:
        selected_ops = []
        for l in xlist:
            selected_ops.append(l[0])
        if sum(np.array(selected_ops) == "none") == len(selected_ops):
            continue
        possible_connections[target_node_idx].append(tuple(xlist))
    print("target_node:{}".format(target_node_idx), len(possible_connections[target_node_idx]))
        
### train while generating random architectures by mutating connections of a target node

## evaluation
network.train()
archs, metric_accs, metric_confidences, metric_sensitivities, metric_robustnesses, metric_step_sims = [], [], [], [], [], []
loader_iter = iter(valid_loader)

for arch_loop in tqdm(range(1000)):
    current_arch_list = []
    for target_cell_idx in range(num_cells):
        target_cell = cells[target_cell_idx]
        ###### random gen
        genotypes = []
        for n in range(1, xargs.max_nodes):
            genotypes.append(random.choice(possible_connections[n]))
        arch = Structure(genotypes)
        target_cell.arch_cache = arch
        current_arch_list.append(arch)
    ###### measure metrics
    try:
        inputs, targets = next(loader_iter)
    except:
        loader_iter = iter(valid_loader)
        inputs, targets = next(loader_iter)
    inputs, targets = inputs.cuda(non_blocking=True), targets.cuda(non_blocking=True)
    valid_acc = acc_confidence_robustness_metrics(network, inputs, targets)
    step_sim = step_sim_metric(network, criterion, inputs, targets)
    archs.append(current_arch_list)
    metric_accs.append(valid_acc)
#     metric_confidences.append(confidence)
#     metric_sensitivities.append(sensitivity)
#     metric_robustnesses.append(robustness)
#     metric_step_sims.append(step_sim)
# rank_accs, rank_confidences, rank_sensitivities, rank_robustnesses, rank_step_sims = stats.rankdata(metric_accs), stats.rankdata(metric_confidences), stats.rankdata(metric_sensitivities), stats.rankdata(metric_robustnesses), stats.rankdata(metric_step_sims)
rank_accs = stats.rankdata(metric_accs)
# l = len(rank_accs)
# rank_agg = np.log(rank_accs/l)+np.log(rank_confidences/l)+np.log(rank_sensitivities/l)+np.log(rank_robustnesses/l)+np.log(rank_step_sims/l)
rank_agg = rank_accs
best_idx = np.argmax(rank_agg)
best_arch, best_acc = archs[best_idx], metric_accs[best_idx]
logger.log("Found best arch with accuracy={:.2f}%".format(best_acc))
for c, a in zip(cells, best_arch):
    c.arch_cache = a
            
best_archs = []
for c in cells:
    best_archs.append(c.arch_cache)
    
torch.save({"model":search_model.state_dict(), "best_archs":best_archs}, os.path.join(xargs.save_dir, "output.pth"))

for m in search_model.modules():
    if isinstance(m, SearchCell):
        logger.log(m.arch_cache)

logger.close()

total number of nodes:60
target_node:1 4
target_node:2 24
target_node:3 124


100%|██████████| 1000/1000 [12:54<00:00,  1.29it/s]

Found best arch with accuracy=73.44%
Structure(4 nodes with |nor_conv_1x1~0|+|none~0|nor_conv_3x3~1|+|none~0|nor_conv_3x3~1|none~2|)
Structure(4 nodes with |avg_pool_3x3~0|+|skip_connect~0|none~1|+|none~0|avg_pool_3x3~1|nor_conv_3x3~2|)
Structure(4 nodes with |skip_connect~0|+|skip_connect~0|none~1|+|nor_conv_1x1~0|none~1|nor_conv_1x1~2|)
Structure(4 nodes with |skip_connect~0|+|skip_connect~0|nor_conv_1x1~1|+|nor_conv_3x3~0|skip_connect~1|none~2|)
Structure(4 nodes with |nor_conv_1x1~0|+|skip_connect~0|nor_conv_3x3~1|+|nor_conv_1x1~0|skip_connect~1|avg_pool_3x3~2|)
Structure(4 nodes with |avg_pool_3x3~0|+|nor_conv_1x1~0|nor_conv_1x1~1|+|nor_conv_1x1~0|avg_pool_3x3~1|skip_connect~2|)
Structure(4 nodes with |nor_conv_1x1~0|+|skip_connect~0|skip_connect~1|+|nor_conv_1x1~0|none~1|nor_conv_1x1~2|)
Structure(4 nodes with |avg_pool_3x3~0|+|nor_conv_3x3~0|none~1|+|nor_conv_3x3~0|skip_connect~1|avg_pool_3x3~2|)
Structure(4 nodes with |skip_connect~0|+|nor_conv_3x3~0|avg_pool_3x3~1|+|skip_conne

In [8]:
# import matplotlib.pyplot as plt

# plt.scatter(rank_confidences,rank_accs)
# plt.show()

# plt.scatter(rank_sensitivities,rank_accs)
# plt.show()

# plt.scatter(rank_robustnesses,rank_accs)
# plt.show()

# plt.scatter(rank_step_sims,rank_accs)
# plt.show()

# Train a found model

In [9]:
trained_output = torch.load(os.path.join(xargs.save_dir, "output.pth"))
print(args)
args.save_dir = os.path.join(xargs.save_dir, "train")
print(args)

Namespace(arch_nas_dataset=None, channel=16, config_path='./MY.config', data_path='../cifar.python', dataset='cifar10', max_nodes=4, num_cells=5, print_freq=200, rand_seed=53064, save_dir='./arch_level-use_pretrain-acc_metric', search_space_name='nas-bench-201', select_num=100, track_running_stats=0, workers=4)
Namespace(arch_nas_dataset=None, channel=16, config_path='./MY.config', data_path='../cifar.python', dataset='cifar10', max_nodes=4, num_cells=5, print_freq=200, rand_seed=53064, save_dir='./arch_level-use_pretrain-acc_metric/train', search_space_name='nas-bench-201', select_num=100, track_running_stats=0, workers=4)


In [10]:
print(config)

Configure(scheduler='cos', LR=0.025, eta_min=0.001, epochs=50, warmup=0, optim='SGD', decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=64, test_batch_size=512, class_num=10, xshape=(1, 3, 32, 32))


In [11]:
logger = prepare_logger(args)

# cifar_train_config_path = "./MY.config"
cifar_train_config_path = "../configs/nas-benchmark/CIFAR.config"
###
train_data, test_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
config = load_config(cifar_train_config_path, {"class_num": class_num, "xshape": xshape}, logger)

train_loader = torch.utils.data.DataLoader(
            train_data,
            batch_size=config.batch_size,
            shuffle=True,
            num_workers=xargs.workers,
            pin_memory=True,)

test_loader = torch.utils.data.DataLoader(
            test_data,
            batch_size=config.batch_size,
            shuffle=True,
            num_workers=xargs.workers,
            pin_memory=True,)

# search_loader, _, valid_loader = get_nas_search_loaders(train_data,
#                                                         valid_data,
#                                                         xargs.dataset,
#                                                         "../configs/nas-benchmark/",
#                                                         (config.batch_size, config.batch_size),
#                                                         xargs.workers)
logger.log("||||||| {:10s} ||||||| Train-Loader-Num={:}, Test-Loader-Num={:}, batch size={:}".format(
            xargs.dataset, len(train_loader), len(test_loader), config.batch_size))
logger.log("||||||| {:10s} ||||||| Config={:}".format(xargs.dataset, config))

search_space = get_search_spaces("cell", xargs.search_space_name)
model_config = dict2config(
    {
        "name": "RANDOM",
        "C": xargs.channel,
        "N": xargs.num_cells,
        "max_nodes": xargs.max_nodes,
        "num_classes": class_num,
        "space": search_space,
        "affine": False,
        "track_running_stats": True, # true for eval
    },
    None,
)
search_model = get_cell_based_tiny_net(model_config)

### load
# trained_output = torch.load(os.path.join(xargs.save_dir, "output.pth"))
# search_model.load_state_dict(trained_output['model'], strict=False)
best_archs = trained_output['best_archs']
i=0
for m in search_model.modules():
    if isinstance(m, SearchCell):
        m.arch_cache = best_archs[i]
        i += 1
for m in network.modules():
    if isinstance(m, SearchCell):
        print(m.arch_cache)
###

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.parameters(), config)

logger.log("w-optimizer : {:}".format(w_optimizer))
logger.log("w-scheduler : {:}".format(w_scheduler))
logger.log("criterion   : {:}".format(criterion))

network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()

last_info, model_base_path, model_best_path = (
    logger.path("info"),
    logger.path("model"),
    logger.path("best"),
)

start_epoch, valid_accuracies, genotypes = 0, {"best": -1}, {}

Main Function with logger : Logger(dir=arch_level-use_pretrain-acc_metric/train, use-tf=False, writer=None)
Arguments : -------------------------------
arch_nas_dataset : None
channel          : 16
config_path      : ./MY.config
data_path        : ../cifar.python
dataset          : cifar10
max_nodes        : 4
num_cells        : 5
print_freq       : 200
rand_seed        : 53064
save_dir         : ./arch_level-use_pretrain-acc_metric/train
search_space_name : nas-bench-201
select_num       : 100
track_running_stats : 0
workers          : 4
Python  Version  : 3.7.13 (default, Mar 29 2022, 02:18:16)  [GCC 7.5.0]
Pillow  Version  : 9.0.1
PyTorch Version  : 1.12.0
cuDNN   Version  : 8302
CUDA available   : True
CUDA GPU numbers : 2
CUDA_VISIBLE_DEVICES : None
Files already downloaded and verified
Files already downloaded and verified
../configs/nas-benchmark/CIFAR.config
Configure(scheduler='cos', eta_min=0.0, epochs=200, warmup=0, optim='SGD', LR=0.1, decay=0.0005, momentum=0.9, nesterov=T

In [12]:
# def search_func_one_arch(xloader, network, criterion, scheduler, w_optimizer, epoch_str, print_freq, logger):
#     data_time, batch_time = AverageMeter(), AverageMeter()
#     base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
#     network.train()
#     end = time.time()
#     for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(
#         xloader
#     ):
#         scheduler.update(None, 1.0 * step / len(xloader))
#         base_targets = base_targets.cuda(non_blocking=True)
#         arch_targets = arch_targets.cuda(non_blocking=True)
#         # measure data loading time
#         data_time.update(time.time() - end)

#         w_optimizer.zero_grad()
#         _, logits = network(base_inputs)
#         base_loss = criterion(logits, base_targets)
#         base_loss.backward()
#         nn.utils.clip_grad_norm_(network.parameters(), 5)
#         w_optimizer.step()
#         # record
#         base_prec1, base_prec5 = obtain_accuracy(
#             logits.data, base_targets.data, topk=(1, 5)
#         )
#         base_losses.update(base_loss.item(), base_inputs.size(0))
#         base_top1.update(base_prec1.item(), base_inputs.size(0))
#         base_top5.update(base_prec5.item(), base_inputs.size(0))

#         # measure elapsed time
#         batch_time.update(time.time() - end)
#         end = time.time()

#         if step % print_freq == 0 or step + 1 == len(xloader):
#             Sstr = (
#                 "*SEARCH* "
#                 + time_string()
#                 + " [{:}][{:03d}/{:03d}]".format(epoch_str, step, len(xloader))
#             )
#             Tstr = "Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})".format(
#                 batch_time=batch_time, data_time=data_time
#             )
#             Wstr = "Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]".format(
#                 loss=base_losses, top1=base_top1, top5=base_top5
#             )
#             logger.log(Sstr + " " + Tstr + " " + Wstr)
#     return base_losses.avg, base_top1.avg, base_top5.avg

def train_func_one_arch(xloader, network, criterion, scheduler, w_optimizer, epoch_str, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()
    for step, (base_inputs, base_targets) in enumerate(
        xloader
    ):
        scheduler.update(None, 1.0 * step / len(xloader))
        base_targets = base_targets.cuda(non_blocking=True)
        # measure data loading time
        data_time.update(time.time() - end)

        w_optimizer.zero_grad()
        _, logits = network(base_inputs)
        base_loss = criterion(logits, base_targets)
        base_loss.backward()
        nn.utils.clip_grad_norm_(network.parameters(), 5)
        w_optimizer.step()
        # record
        base_prec1, base_prec5 = obtain_accuracy(
            logits.data, base_targets.data, topk=(1, 5)
        )
        base_losses.update(base_loss.item(), base_inputs.size(0))
        base_top1.update(base_prec1.item(), base_inputs.size(0))
        base_top5.update(base_prec5.item(), base_inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if step % print_freq == 0 or step + 1 == len(xloader):
            Sstr = (
                "*SEARCH* "
                + time_string()
                + " [{:}][{:03d}/{:03d}]".format(epoch_str, step, len(xloader))
            )
            Tstr = "Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})".format(
                batch_time=batch_time, data_time=data_time
            )
            Wstr = "Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]".format(
                loss=base_losses, top1=base_top1, top5=base_top5
            )
            logger.log(Sstr + " " + Tstr + " " + Wstr)
    return base_losses.avg, base_top1.avg, base_top5.avg

def valid_func_one_arch(xloader, network, criterion):
    data_time, batch_time = AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.eval()
    end = time.time()
    with torch.no_grad():
        for step, (arch_inputs, arch_targets) in enumerate(xloader):
            arch_targets = arch_targets.cuda(non_blocking=True)
            # measure data loading time
            data_time.update(time.time() - end)
            # prediction

#             network.module.random_genotype_per_cell(True)
            _, logits = network(arch_inputs)
            arch_loss = criterion(logits, arch_targets)
            # record
            arch_prec1, arch_prec5 = obtain_accuracy(
                logits.data, arch_targets.data, topk=(1, 5)
            )
            arch_losses.update(arch_loss.item(), arch_inputs.size(0))
            arch_top1.update(arch_prec1.item(), arch_inputs.size(0))
            arch_top5.update(arch_prec5.item(), arch_inputs.size(0))
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
    return arch_losses.avg, arch_top1.avg, arch_top5.avg

In [13]:
start_time, search_time, epoch_time, total_epoch = (
    time.time(),
    AverageMeter(),
    AverageMeter(),
    config.epochs + config.warmup,
)
for epoch in range(0, total_epoch):
    w_scheduler.update(epoch, 0.0)
    need_time = "Time Left: {:}".format(
        convert_secs2time(epoch_time.val * (total_epoch - epoch), True)
    )
    epoch_str = "{:03d}-{:03d}".format(epoch, total_epoch)
    logger.log(
        "\n[Search the {:}-th epoch] {:}, LR={:}".format(
            epoch_str, need_time, min(w_scheduler.get_lr())
        )
    )

    # selected_arch = search_find_best(valid_loader, network, criterion, xargs.select_num)
    search_w_loss, search_w_top1, search_w_top5 = train_func_one_arch(
        train_loader,
        network,
        criterion,
        w_scheduler,
        w_optimizer,
        epoch_str,
        xargs.print_freq,
        logger,
    )
    search_time.update(time.time() - start_time)
    logger.log(
        "[{:}] searching : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%, time-cost={:.1f} s".format(
            epoch_str, search_w_loss, search_w_top1, search_w_top5, search_time.sum
        )
    )
    valid_a_loss, valid_a_top1, valid_a_top5 = valid_func_one_arch(
        test_loader, network, criterion
    )
    logger.log(
        "[{:}] evaluate  : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%".format(
            epoch_str, valid_a_loss, valid_a_top1, valid_a_top5
        )
    )
    
    # check the best accuracy
    valid_accuracies[epoch] = valid_a_top1
    if valid_a_top1 > valid_accuracies["best"]:
        valid_accuracies["best"] = valid_a_top1
        find_best = True
    else:
        find_best = False

    # save checkpoint
    save_path = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(xargs),
            "search_model": search_model.state_dict(),
            "w_optimizer": w_optimizer.state_dict(),
            "w_scheduler": w_scheduler.state_dict(),
            "genotypes": genotypes,
            "valid_accuracies": valid_accuracies,
        },
        model_base_path,
        logger,
    )
    last_info = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(args),
            "last_checkpoint": save_path,
        },
        logger.path("info"),
        logger,
    )
    if find_best:
        logger.log(
            "<<<--->>> The {:}-th epoch : find the highest validation accuracy : {:.2f}%.".format(
                epoch_str, valid_a_top1
            )
        )
        copy_checkpoint(model_base_path, model_best_path, logger)
    if api is not None:
        logger.log("{:}".format(api.query_by_arch(genotypes[epoch], "200")))
    # measure elapsed time
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

logger.close()


[Search the 000-200-th epoch] Time Left: [00:00:00], LR=0.1
*SEARCH* [2022-11-08 04:28:00] [000-200][000/196] Time 0.44 (0.44) Data 0.34 (0.34) Base [Loss 2.359 (2.359)  Prec@1 11.33 (11.33) Prec@5 46.09 (46.09)]
*SEARCH* [2022-11-08 04:28:29] [000-200][195/196] Time 0.13 (0.15) Data 0.00 (0.00) Base [Loss 1.349 (1.632)  Prec@1 56.25 (39.37) Prec@5 92.50 (88.30)]
[000-200] searching : loss=1.63, accuracy@1=39.37%, accuracy@5=88.30%, time-cost=29.3 s
[000-200] evaluate  : loss=1.75, accuracy@1=38.15%, accuracy@5=90.73%
save checkpoint into arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth
save checkpoint into arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth
<<<--->>> The 000-200-th epoch : find the highest validation accuracy : 38.15%.
copy the file from arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth into arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-best.pth

[Search the 001-200-th epoch] Time Left: [01

*SEARCH* [2022-11-08 04:31:24] [007-200][000/196] Time 0.46 (0.46) Data 0.35 (0.35) Base [Loss 0.704 (0.704)  Prec@1 76.17 (76.17) Prec@5 99.61 (99.61)]
*SEARCH* [2022-11-08 04:31:44] [007-200][195/196] Time 0.09 (0.10) Data 0.00 (0.00) Base [Loss 0.775 (0.660)  Prec@1 77.50 (77.11) Prec@5 97.50 (98.66)]
[007-200] searching : loss=0.66, accuracy@1=77.11%, accuracy@5=98.66%, time-cost=201.3 s
[007-200] evaluate  : loss=1.00, accuracy@1=67.57%, accuracy@5=97.76%
Find arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth
Find arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth

[Search the 008-200-th epoch] Time Left: [01:13:42], LR=0.0996057350657239
*SEARCH* [2022-11-08 04:31:47] [008-200][000/196]

*SEARCH* [2022-11-08 04:35:25] [015-200][000/196] Time 0.50 (0.50) Data 0.40 (0.40) Base [Loss 0.526 (0.526)  Prec@1 82.42 (82.42) Prec@5 99.22 (99.22)]
*SEARCH* [2022-11-08 04:35:44] [015-200][195/196] Time 0.07 (0.10) Data 0.00 (0.00) Base [Loss 0.630 (0.516)  Prec@1 76.25 (82.13) Prec@5 97.50 (99.15)]
[015-200] searching : loss=0.52, accuracy@1=82.13%, accuracy@5=99.15%, time-cost=415.1 s
[015-200] evaluate  : loss=0.75, accuracy@1=74.70%, accuracy@5=98.24%
Find arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth
Find arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth

[Search the 016-200-th epoch] Time Left: [01:10:54], LR=0.09842915805643156
*SEARCH* [2022-11-08 04:35:48] [016-200][000/196

[023-200] evaluate  : loss=0.70, accuracy@1=75.78%, accuracy@5=98.77%
Find arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth
Find arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth

[Search the 024-200-th epoch] Time Left: [01:04:11], LR=0.09648882429441258
*SEARCH* [2022-11-08 04:39:15] [024-200][000/196] Time 0.45 (0.45) Data 0.34 (0.34) Base [Loss 0.459 (0.459)  Prec@1 85.16 (85.16) Prec@5 99.22 (99.22)]
*SEARCH* [2022-11-08 04:39:34] [024-200][195/196] Time 0.07 (0.10) Data 0.00 (0.00) Base [Loss 0.359 (0.458)  Prec@1 90.00 (84.17) Prec@5 97.50 (99.39)]
[024-200] searching : loss=0.46, accuracy@1=84.17%, accuracy@5=99.39%, time-cost=611.6 s
[024-200] evaluate  : loss=0.70, accuracy@1=76.9

*SEARCH* [2022-11-08 04:43:06] [032-200][195/196] Time 0.13 (0.15) Data 0.00 (0.00) Base [Loss 0.283 (0.434)  Prec@1 91.25 (84.98) Prec@5 100.00 (99.47)]
[032-200] searching : loss=0.43, accuracy@1=84.98%, accuracy@5=99.47%, time-cost=793.5 s
[032-200] evaluate  : loss=0.65, accuracy@1=79.01%, accuracy@5=98.77%
Find arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth
Find arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth

[Search the 033-200-th epoch] Time Left: [01:35:07], LR=0.09343157572190958
*SEARCH* [2022-11-08 04:43:11] [033-200][000/196] Time 0.54 (0.54) Data 0.44 (0.44) Base [Loss 0.355 (0.355)  Prec@1 88.28 (88.28) Prec@5 99.61 (99.61)]
*SEARCH* [2022-11-08 04:43:30] [033-200][195/19

...

[166-200] evaluate  : loss=0.34, accuracy@1=89.88%, accuracy@5=99.70%
Find arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth
Find arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth
<<<--->>> The 166-200-th epoch : find the highest validation accuracy : 89.88%.
Find arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-best.pth exist, delete is at first before saving
copy the file from arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth into arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-best.pth

[Search the 167-200-th epoch] Time Left: [00:16:03], LR=0.006568424278090435
*SEARCH* [2022-11-08 05:44:11] [167-200][000/196] Time 0.50 (0.50) Data 0.38

save checkpoint into arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth
Find arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth
<<<--->>> The 173-200-th epoch : find the highest validation accuracy : 90.42%.
Find arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-best.pth exist, delete is at first before saving
copy the file from arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth into arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-best.pth

[Search the 174-200-th epoch] Time Left: [00:12:50], LR=0.004112268715800955
*SEARCH* [2022-11-08 05:47:26] [174-200][000/196] Time 0.50 (0.50) Data 0.40 (0.40) Base [Loss 0.069 (0.069)  Prec@1 96.88 (96.88) Prec@5 100.00 (100.00)]
*SEARCH* [2022-11-08 05:47:45] [174-200][195/196] Time 0.09 (0.10) Data 0.00 (0.00) Base [Loss 0.062 (0.062) 

*SEARCH* [2022-11-08 05:50:57] [181-200][195/196] Time 0.13 (0.15) Data 0.00 (0.00) Base [Loss 0.061 (0.046)  Prec@1 97.50 (98.66) Prec@5 100.00 (99.99)]
[181-200] searching : loss=0.05, accuracy@1=98.66%, accuracy@5=99.99%, time-cost=4332.3 s
[181-200] evaluate  : loss=0.33, accuracy@1=90.95%, accuracy@5=99.71%
Find arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth
Find arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth
<<<--->>> The 181-200-th epoch : find the highest validation accuracy : 90.95%.
Find arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-best.pth exist, delete is at first before saving
copy the file from arch_level-use_pretrain-acc_metric/train/checkpoint/seed-530

*SEARCH* [2022-11-08 05:54:13] [189-200][000/196] Time 0.67 (0.67) Data 0.46 (0.46) Base [Loss 0.028 (0.028)  Prec@1 100.00 (100.00) Prec@5 100.00 (100.00)]
*SEARCH* [2022-11-08 05:54:32] [189-200][195/196] Time 0.15 (0.10) Data 0.00 (0.00) Base [Loss 0.049 (0.037)  Prec@1 98.75 (99.07) Prec@5 100.00 (100.00)]
[189-200] searching : loss=0.04, accuracy@1=99.07%, accuracy@5=100.00%, time-cost=4516.1 s
[189-200] evaluate  : loss=0.33, accuracy@1=90.97%, accuracy@5=99.69%
Find arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth
Find arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth

[Search the 190-200-th epoch] Time Left: [00:03:46], LR=0.0006155829702431171
*SEARCH* [2022-11-08 05:54:36] [190-20

*SEARCH* [2022-11-08 05:58:00] [198-200][000/196] Time 0.66 (0.66) Data 0.36 (0.36) Base [Loss 0.024 (0.024)  Prec@1 99.61 (99.61) Prec@5 100.00 (100.00)]
*SEARCH* [2022-11-08 05:58:26] [198-200][195/196] Time 0.07 (0.13) Data 0.00 (0.00) Base [Loss 0.100 (0.033)  Prec@1 96.25 (99.16) Prec@5 100.00 (100.00)]
[198-200] searching : loss=0.03, accuracy@1=99.16%, accuracy@5=100.00%, time-cost=4720.4 s
[198-200] evaluate  : loss=0.32, accuracy@1=91.12%, accuracy@5=99.73%
Find arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/checkpoint/seed-53064-basic.pth
Find arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth exist, delete is at first before saving
save checkpoint into arch_level-use_pretrain-acc_metric/train/seed-53064-last-info.pth

[Search the 199-200-th epoch] Time Left: [00:00:29], LR=6.168375916970615e-06
*SEARCH* [2022-11-08 05:58:29] [199-200]

In [14]:
best_archs

[Structure(4 nodes with |nor_conv_1x1~0|+|none~0|nor_conv_3x3~1|+|none~0|nor_conv_3x3~1|none~2|),
 Structure(4 nodes with |avg_pool_3x3~0|+|skip_connect~0|none~1|+|none~0|avg_pool_3x3~1|nor_conv_3x3~2|),
 Structure(4 nodes with |skip_connect~0|+|skip_connect~0|none~1|+|nor_conv_1x1~0|none~1|nor_conv_1x1~2|),
 Structure(4 nodes with |skip_connect~0|+|skip_connect~0|nor_conv_1x1~1|+|nor_conv_3x3~0|skip_connect~1|none~2|),
 Structure(4 nodes with |nor_conv_1x1~0|+|skip_connect~0|nor_conv_3x3~1|+|nor_conv_1x1~0|skip_connect~1|avg_pool_3x3~2|),
 Structure(4 nodes with |avg_pool_3x3~0|+|nor_conv_1x1~0|nor_conv_1x1~1|+|nor_conv_1x1~0|avg_pool_3x3~1|skip_connect~2|),
 Structure(4 nodes with |nor_conv_1x1~0|+|skip_connect~0|skip_connect~1|+|nor_conv_1x1~0|none~1|nor_conv_1x1~2|),
 Structure(4 nodes with |avg_pool_3x3~0|+|nor_conv_3x3~0|none~1|+|nor_conv_3x3~0|skip_connect~1|avg_pool_3x3~2|),
 Structure(4 nodes with |skip_connect~0|+|nor_conv_3x3~0|avg_pool_3x3~1|+|skip_connect~0|avg_pool_3x3~1|